In [5]:
!conda activate base
# Required Importscmd
import cv2 as cv2
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.figure as figure
import matplotlib.patches as patches
from ast import literal_eval
import pickle
import numpy as np
import pandas as pd
from PIL import Image
import os 

#pip install jupyter_innotater
from jupyter_innotater import *

In [6]:
# Image general definitions
filename = "RUBEM_BERTA_2_60_6_mars-small128" ## name of the csv file
videoname = "RUBEM_BERTA_2" ## name of the video


data = pd.read_csv(f"./data/{filename}.csv", sep=";", 
                   names=[
                   "track_id",
                    "bbox",
                    "class",
                    "frame"],
                    converters={"bbox": literal_eval}) ## read the csv file and literal eval as list the bbox field
vidcap = cv2.VideoCapture(f"./data/{videoname}.mp4") ## open the video file

WIDTH = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)) ## get the with of the frame
HEIGHT = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)) ## get the height of the frame 
my_dpi = 96 ## pixel density of the current monitor - to ensure the size of image complies with the pixel size of the screen
MIN_DISTANCE = 100 ## minimum distance threshold to filter the data

fps = vidcap.get(cv2.CAP_PROP_FPS) ## get the fps of the video
# Get the total numer of frames in the video.
frame_count = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

In [7]:
## capture the image at 1s in the video - ensure that the image is clear and stable.
vidcap.set(cv2.CAP_PROP_POS_MSEC, 1000)  # change the vidcap position
_ , image = vidcap.read() ## buffer a call
_ , image = vidcap.read() ## then capture the image
cv2.imwrite("first_frame.png", image) ## save the image (first frame for defining the entrances and outputs)

## Get the center of the bounding boxes based on the bbox data
## (0,0 on top left corner)
## (WIDTH,HEIGTH on bottom right corner)
## xc = x1 + x2 / 2
## yc = HEIGTH - (y1 + y2)/2

data["x"] = (data.bbox.str[0] +
             data.bbox.str[2]) / 2

data["y"] = -(data.bbox.str[1] +
              data.bbox.str[3]) / 2 + HEIGHT

## filter data by track_id
grouped = data.groupby("track_id") # group the data by the object ID 

In [10]:
#_id = 2 
# model = "vric"
_time = 0 # Initial time in frames
_delta = 300 # frame delta - duration of the logging in the image

plt.figure(figsize=(WIDTH/my_dpi, HEIGHT/my_dpi), dpi=my_dpi) # start a figure plot
segment = data[(data.frame < _time + _delta) & (data.frame > _time)] # segment of data that we are interested in
#segment = data
plt.scatter(segment.x, segment.y, c=segment.track_id, cmap='jet', s=12) # plot the scatter with the colormap being the frame index
plt.xlabel("X")
plt.ylabel("Y")
#plt.colorbar(label="Frame Index")

plt.tight_layout() # tight layout
plt.savefig(f"./scatter_{filename}_{_time}_{_delta}.png", dpi=96) # save the figure

In [11]:
def plot_arrow(data, color):
    plt.scatter(data["x"].values, data["y"].values,
                c='blue', marker='o', s=10, zorder=3) ## size and z position, color blue, plot the raw centers for the entries
    plt.scatter(data["last_x"].values,
                data["last_y"].values, c='red', marker='o', s=10, zorder=2) ## plot output points, with color red

    plt.quiver(data["x"], data["y"], ((data["last_x"] - data["x"])), ((data["last_y"] - data["y"])),
                                 angles="xy", scale_units="xy", scale=1, width=0.0015, color=color) ## plot the arrows

In [12]:
data = pd.DataFrame() ## Constructing DataFrame from a dictionary 

last_3x = []
last_3y = []
last_bboxes = []
avg_distance = []

## random auxiliary data
## used to rough estimate the dx and dy values (velocities) in pixel/frame for the tracks

for _, group in grouped: ## for all track ids
    last_3x.append(group.tail(3).x.values) ## get the last 3 x points - outputs
    last_3y.append(group.tail(3).y.values) ## get the last 3 y points - outputs
    last_bboxes.append(list(group.tail(3).bbox.values)) ## get the last 3 bboxes - outputs
    data = data.append(group.head(1)) ## get only the first group for any given track id -> should have only a single entry for track id)
## output file from DeepSORT: track_ID, [x1,y1,x2,y2],class, frame_#

## assign required data and convert between types.
## the groupby is not inplace, so we add the required data from the created lists.
data["last_3x"] = last_3x
data["last_3y"] = last_3y
data["last_x"] = data["last_3x"].str[2]
data["last_y"] = data["last_3y"].str[2]
data["last_bboxes"] = last_bboxes

data["distance"] = ((data.x - data.last_x)**2 +
                    ((data.y - data.last_y)**2))**0.5
## cartesian distance
## d = sqrt ((x1-x2)² + (y1-y2)²)

## filtering data based on the minimun distance in pixels
data = data[data["distance"] > MIN_DISTANCE]

## estimate of the dx and dy within 3 frames (delta position / delta frames = x - x` / 3)
data["est_dx"] = (data.last_3x.str[2] - data.last_3x.str[0])/3
data["est_dy"] = (data.last_3y.str[2] - data.last_3y.str[0])/3

## generate the figure with the required size in pixels to fit the dpi
## tis will be the reading from the first frame
fig = plt.figure(figsize=(WIDTH/my_dpi, HEIGHT/my_dpi), dpi=my_dpi)
img = plt.imread("first_frame.png")
plt.imshow(img, extent=[0, WIDTH, 0, HEIGHT])

## plot arrows with different colors for each class
plot_arrow(data.loc[data['class'] == 'car'], 'black')
plot_arrow(data.loc[data['class'] == 'motorcycle'], 'green')
plot_arrow(data.loc[data['class'] == 'bus'], 'orange')
plot_arrow(data.loc[data['class'] == 'truck'], 'purple')

## remove the axis before saving the images. Also, remove some other formatting that messes it up
plt.axis('off')
plt.gca().set_axis_off()
plt.subplots_adjust(top=1, bottom=0, right=1, left=0,
                    hspace=0, wspace=0)
plt.margins(0, 0)

plt.savefig(f"detected_arrows_{filename}.png", figsize=(
    WIDTH/my_dpi, HEIGHT/my_dpi), dpi=my_dpi)

plt.savefig(f"composed_image_{filename}.png",
            bbox_inches='tight', pad_inches=0, dpi=my_dpi, figsize=(WIDTH/my_dpi, HEIGHT/my_dpi))

#im = cv2.imread(f"composed_image_{filename}.png") ## read the image to select the RoIs
#rois = cv2.selectROIs("RoI Selector", im) ## select the ROIS in the frame 
## Selects ROIs on the given image. Function creates a window and allows user to select a ROIs using mouse. 
##Controls: use `space` or `enter` to finish current selection and start a new one, use `esc` to terminate 
##multiple ROI selection process. 
# roi = (x1,y1,w,h) (top left)

# cv2.destroyAllWindows()


In [61]:
targets_bboxes = np.zeros((1, 8, 4), dtype='int') # (x,y,w,h) for each region ## DOES NOT WORK ON COLAB!!
SCALE = 2 

Innotater( 
    ImageInnotation([f"composed_image_{filename}.png"], path='./', width= int(WIDTH/SCALE), height=int(HEIGHT/SCALE)), 
    [
        RepeatInnotation(
            (BoundingBoxInnotation, targets_bboxes),
             max_repeats=8, min_repeats=1
        )
    ]
)

Innotater(children=(HBox(children=(VBox(children=(ImagePad(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00…

In [63]:
rois = []
for bbox in targets_bboxes[0]: ## for bboxes
    if (np.any(bbox)): ## if has size
        rois.append(bbox) ## is a valid roi

## generate the figure with the required size in pixels to fit the dpi
## tis will be the reading from the first frame
fig = plt.figure(figsize=(WIDTH/my_dpi, HEIGHT/my_dpi), dpi=my_dpi)
img = plt.imread(f"composed_image_{filename}.png")
plt.imshow(img, extent=[0, WIDTH, 0, HEIGHT])        
        
## convert the data given by the roi selector to a more usual format
## roi selector gives it in the same way with 0,0 being the top left, whereas 
## matplotlib uses its 0,0 being on the bottom left.
#rectangle_coords = [[roi[0], HEIGHT - roi[1] -
#                     roi[3], roi[2], roi[3]] for roi in rois] ## get the roi box coordinates
rectangle_coords = [[roi[0], HEIGHT - roi[1] - roi[3],
                     roi[2], HEIGHT - roi[1]] for roi in rois] ## get the roi box coordinates

## identify the rectangles
for index, rect in enumerate(rectangle_coords, start=1):
    print(f"Rectangle {index}: {rect}")

## generate the rectangles to be plotted based on the data given by the RoI Selector.
rectangles = [patches.Rectangle((coord[0], coord[1]), abs(coord[2]-coord[0]), abs(coord[3]-coord[1]),
                                color="lime", linewidth=2, fill=False) for coord in rectangle_coords]

## generate a dictionary for the plot of the rectangles, which will be plotted with their numbers.
rects = dict(enumerate(rectangles, start=1))

ax = plt.gca() ## get the current axis to make te proper plot
for r in rects:
    ax.add_artist(rects[r])
    rx, ry = rects[r].get_xy()
    cx = rx + rects[r].get_width()/2.0
    cy = ry + rects[r].get_height()/2.0

    ax.annotate(r, (cx, cy), color='lime', weight='bold',
                fontsize=72, ha='center', va='center')

plt.axis('off')
plt.gca().set_axis_off()
plt.subplots_adjust(top=1, bottom=0, right=1, left=0,
                    hspace=0, wspace=0)
plt.margins(0, 0)

## saves the figure with the rectangles drawn over it
plt.savefig(f"regions_{filename}.png",
            bbox_inches='tight', pad_inches=0)

## assistant func to verify if a point is inside the a bounding box (of the regions)
def verifyIfInRect(x, y):
    for number, bbox in enumerate(rectangle_coords, start=1):
        # print(bbox[0] , (bbox[0] + bbox[2]), bbox[1] , (bbox[1] + bbox[3]))
        if bbox[0] <= x <= (bbox[0] + bbox[2]) and bbox[1] <= y <= (bbox[1] + bbox[3]):
            return number
    return 0 ## if not inside any bbox

## by using data.apply, we can apply our function to determine the rectangle the 
## x,y points are in // for entry and exit points.

data["entry_box"] = data.apply(lambda row: verifyIfInRect(
    row["x"], row["y"]), axis=1)

data["exit_box"] = data.apply(lambda row: verifyIfInRect(
    row["last_3x"][-1], row["last_3y"][-1]), axis=1)

## save the dataframe to CSV (issues with lists, like the bbox)
## but also save as an object - which is the case of the pickle file
## to which we can have the same object opened on another folder without
## having to re-convert the list
data.to_csv(f"classified_data_{filename}.csv")
pickle.dump(data,open(f"classified_data_{filename}.pkl","wb"))

C:\Users\luizg\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Rectangle 1: [856, 43, 420, 465]
Rectangle 2: [32, 249, 382, 565]
